In [ ]:
from urllib import request, parse
from http.cookiejar import CookieJar
import jq
import getpass
import netrc
import requests
import json
import os
import boto3
import io
from urllib.parse import urlparse
import sys  
!{sys.executable} -m pip install --user jsonlines
import jsonlines


In [ ]:
def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

In [ ]:
setup_earthdata_login_auth('urs.earthdata.nasa.gov')
response = requests.get("https://cmr.earthdata.nasa.gov/search/granules.umm_json?concept_id=C2021957295-LPCLOUD&page_size=1000")
results = json.loads(response.content)

In [ ]:
session = boto3.Session(profile_name='sandbox')
client = session.client('s3')

items = results["items"]
for item in items:
    
    links = item['umm']["RelatedUrls"]
    for link in links:
        if link['Type'] == "GET RELATED VISUALIZATION":
            if link['URL'].startswith('https://'):
                url = link['URL']
                print("HTTPS URL for data: " + url)
                key = os.path.basename(url)
                print(key)
                # Download file
                file = requests.get(url)
                response = client.put_object(
                    Bucket='dug-cloud-training',
                    Key=key,
                    Body=file.content
                )
                break;